# 🤨 How do people feel about Hasanabi's political views?

A little of backstory first. I'm a 2nd year uni student on a computer science course. One of my classes is computational intelligence, where we are introduced to the world of machine learning algorithms and ai. Our last assignment of the semester is to use a Twitter scraper and analyze feelings of people engaging in some topic. I wanted to make this interesting so I chose something that would make me finish this. When I asked my friend on suggestions what should I do he suggested me Hasan.

He is a popular twitch streamer, internet personality, ¿socialist activist? (for sure someone with socialistic views). He has an online following and is overall liked by the internet crowd, I too engaged with his works (mainly entertainment). Because of his political and economic views he could be named someone that raises a lot of noise and makes people feel a certain way so he is a perfect match for a subject of this project!

What we will do is we will download a lot of Mr. Piker's tweets and people's comments under those posts. We will put them through language analyzing algorithms and see how people feel about his political views.

DISCLAIMER: Before we continue I'd like to add that this is mainly for entertainment and you SHOULD NOT base any thoughts on it. You should remember that this was made by a uni CS student. ML (machine learning) is also something flawed:

- it generates slightly different outcomes every time it's run
- is based on math
- is something problematic because it's hard to explain a computer how does "feelings" and human language work

But we can laugh about what will turn out of this. Maybe I could even get Hasan to show it on stream? So without further ado let's continue!


In [2]:
# Importing python libraries nothing special
import pandas as pd
import snscrape.modules.twitter as sntwitter
from snscrape.modules.twitter import Tweet
from datetime import datetime
from datetime import timedelta
from typing import Union
import nltk
from dateutil.parser import parse as date_parse
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import matplotlib.pyplot as plt
import text2emotion as te
from wordcloud import WordCloud
import json
import html
import re


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hubertkowalski/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hubertkowalski/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hubertkowalski/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Getting the data

We will now proceed to the code that will make a search request on twitter site and scrape everything we want from it's result. For that we will use snscraper's twitter module.


In [3]:
# Saving Hasan's twitter handle for later use
HASAN_TWITTER_HANDLE = "hasanthehun"


In [4]:
# Helper function that gets parameters from us parses it into a valid twitter api query
# gets tweets and returns them as a list
def get_hasan_tweets(start_date: datetime, end_date: datetime, num_tweets: int = -1) -> list[Tweet]:
    tweets: list[Tweet] = []

    query = f"from:{HASAN_TWITTER_HANDLE} since:{start_date.strftime('%Y-%m-%d')} until:{end_date.strftime('%Y-%m-%d')}"
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        tweets.append(tweet)

        if len(tweets) == num_tweets:
            break

    return tweets

Let's make a test request to twitter!


In [121]:
tweets = get_hasan_tweets(
    date_parse('2023-03-05'), date_parse('2023-03-06'), num_tweets=5)

In [122]:
tweets[0].rawContent


'@MoistCr1TiKaL 4 companies practically own their own territories and for some reason we can’t hold them accountable despite the billions we have spent building and maintaining the infastructure they now own 🤔'

Yay it works! Now for the less fun part.

Because, at the time of me writing this, Twitter has turned off, literally two days ago, a search filter on their site that is STILL in their API query reference in their docs. I cannot use conversation_id filter to search only for replies to a tweet. We need to get creative. (THANK YOU ELON!)

We will get every reply to hasan and then match them to their respectable "parent" tweet.


In [5]:
def get_replies_to_hasan(start_date: datetime, end_date: datetime, num_tweets: int = -1) -> list[Tweet]:
    replies: list[Tweet] = []

    query = f"to:{HASAN_TWITTER_HANDLE} since:{start_date.strftime('%Y-%m-%d')} until:{end_date.strftime('%Y-%m-%d')} filter:replies"
    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        replies.append(tweet)

        if len(replies) == num_tweets:
            break

    return replies


In [124]:
replies = get_replies_to_hasan(date_parse(
    '2023-03-05'), date_parse('2023-03-06'), num_tweets=5)

In [125]:
for reply in replies[:5]:
    print(reply.rawContent)


@hasanthehun Why are you talking shit already the game isn’t even out
@hasanthehun Good, it’s about time
@hasanthehun I'm sad there won't be any more IRL Japan streams - loved them so much @hasanthehun
@hasanthehun I stg the accent at the end racist as hell
@hasanthehun Marginalized populations? The alphabet bunch is calling all the shots in the democrat party right now.


Our filters work. So now we're going to get every Hasan's tweet and every reply. It'll be a lot of data.


In [6]:
# Helper function that will get only the data we want from a tweet
# We don't want to save data that we won't be using
def parse_post(tweet: Tweet):
    parsed_tweet = {}
    parsed_tweet['id'] = tweet.id
    parsed_tweet['url'] = tweet.url
    parsed_tweet['date'] = tweet.date.strftime('%Y-%m-%d %H:%M:%S')
    parsed_tweet['rawContent'] = tweet.rawContent
    parsed_tweet['renderedContent'] = tweet.renderedContent
    parsed_tweet['conversationId'] = tweet.conversationId
    parsed_tweet['lang'] = tweet.lang
    parsed_tweet['inReplyToTweetId'] = tweet.inReplyToTweetId

    return parsed_tweet

In [127]:
tweets = get_hasan_tweets(date_parse('2020-01-01'), datetime.now())

Unavailable user in card on tweet 1480964717443047426
User 27073265 not found in user refs in card on tweet 1480964717443047426
Stopping after 20 empty pages


In [128]:
# parsing tweets
parsed_tweets = []
for tweet in tweets:
    parsed_tweets.append(parse_post(tweet))


# Saving them to a json file
with open("hasan_tweets.json", "w") as file:
    json.dump([parsed_tweet for parsed_tweet in parsed_tweets], file)

In [129]:
replies = get_replies_to_hasan(date_parse('2020-01-01'), datetime.now())

Unavailable user in card on tweet 1660148601408966658
User 25662595 not found in user refs in card on tweet 1660148601408966658
Could not translate t.co card URL on tweet 1637852460390686721
Stopping after 20 empty pages


In [130]:
# Parsing replies
parsed_replies = []
for reply in replies:
    parsed_replies.append(parse_post(reply))

# Saving them to a json file
with open("hasan_replies.json", "w") as file:
    json.dump([parsed_reply for parsed_reply in parsed_replies], file)

Out of that we got 11.7 thousand tweets from hasan (he do post a lot) and 84.5 thousand replies at him. This is not a humongous amount of data but is big enough for our experiments. I decided to get all of the tweets from 1st of January 2020 until 27th of May 2023 (the day of writing this). This should include many interesting events. COVID-19, BLM protests, US election and recent stuff, probably comments about current post COVID economic state of the US.

Because of those interesting events and hope that Hasan engaged in the conversation on those topics should stir up some conversations and most importantly extreme feelings (I'd like to remind you that we are still talking about twitter).

Our data also includes things like Hasan announcing that he will be live soon, replies to his work friends and other non-political topics. Those tweets should encourage people to reply in a positive way and this would make our data with a tendency to lean in to more positive feelings.

We will see what will turn out of this.

Our data look something like this:

```json
{
  "id": 1662165779498795010, // id of the tweet
  "url": "https://twitter.com/hasanthehun/status/1662165779498795010", // tweets url
  "date": "2023-05-26 18:36:18", // date in a string format
  "rawContent": "DEBT CEILING QUESTIONS ANSWERED W/ @ddayen + HOGWATCH...", // raw content of the tweet it's normalized to ASCII
  "renderedContent": "DEBT CEILING QUESTIONS ANSWERED W/ @ddayen + HOGWATCH...", // content how it's being showed in a browser
  "conversationId": 1662165779498795010, // id of the conversation, if it's a normal tweet it is its id
  "lang": "en", // lang that has been detected by twitter
  "inReplyToTweetId": null // if it's a reply here it shows to which tweet
}
```


In [7]:
# Loading normalized tweets and replies
hasan_tweets = pd.read_json('hasan_tweets.json')

print(hasan_tweets.shape)

(11714, 8)


In [8]:
replies_to_hasan = pd.read_json('hasan_replies.json')

print(replies_to_hasan.shape)

(84488, 8)


## Preprocessing

Before we continue we need to make a little bit of preprocessing. We need to remove any duplicates from the replies, replies with links (they might be bots promoting scam), and empty tweets.


In [9]:
# Removing duplicates
replies_to_hasan.drop_duplicates(subset=['id', "rawContent"], inplace=True)

# Removing tweets that contain @@@@@@@@ or http or https
replies_to_hasan = replies_to_hasan[~replies_to_hasan['rawContent'].str.contains(
    '@@@@@@@@')]
replies_to_hasan = replies_to_hasan[~replies_to_hasan['rawContent'].str.contains(
    'http://')]
replies_to_hasan = replies_to_hasan[~replies_to_hasan['rawContent'].str.contains(
    'https://')]

# Printing shape of the data frame (how much data we are left with)
print(replies_to_hasan.shape)


(70540, 8)


We also have to do something with html codes. What are they? You see everything on the web is written in html. This is a markup language that eases formatting text. Html has some that do some things in it but we also want to put them into the text. Those characters are "<", ">" and others like "/". So they are not encoded into html code, which is what we have in our data. What we have to do is map those symbols into ASCII.

Also we gotta do something with emojis. They also are represented in a string of characters like "\ud83d\ude14" which is translated to "😔".


In [10]:
# This is a helper function that will remove all emojis from a string
def remove_emojis(text: str) -> str:
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


We also want to remove any twitter handles, since they also doesn't contribute much to the overall meaning of the sentence.


In [11]:
def remove_twitter_handles(text: str) -> str:
    return re.sub(r'@\w+', '', text)

Now we will apply this function and unescape any html entities.


In [12]:
hasan_tweets['rawContent'] = hasan_tweets['rawContent'].apply(
    html.unescape).apply(remove_emojis).apply(remove_twitter_handles)

hasan_tweets['renderedContent'] = hasan_tweets['renderedContent'].apply(
    html.unescape).apply(remove_emojis).apply(remove_twitter_handles)


In [13]:
print(hasan_tweets.head())

                    id                                                url   
0  1662165779498795010  https://twitter.com/hasanthehun/status/1662165...  \
1  1661955536580132864  https://twitter.com/hasanthehun/status/1661955...   
2  1661874461170348035  https://twitter.com/hasanthehun/status/1661874...   
3  1661800157409873920  https://twitter.com/hasanthehun/status/1661800...   
4  1661784494301646848  https://twitter.com/hasanthehun/status/1661784...   

                 date                                         rawContent   
0 2023-05-26 18:36:18  DEBT CEILING QUESTIONS ANSWERED W/  + HOGWATCH...  \
1 2023-05-26 04:40:52   if you tell them you watch me they will elimi...   
2 2023-05-25 23:18:42                             this is a gay man btw    
3 2023-05-25 18:23:26  RON DESASTEROUS CAMPAIGN LAUNCH, OATHKEEPER LE...   
4 2023-05-25 17:21:12                   we are never getting healthcare.   

                                     renderedContent       conversationId   
0  

In [14]:
replies_to_hasan['rawContent'] = replies_to_hasan['rawContent'].apply(
    html.unescape).apply(remove_emojis).apply(remove_twitter_handles)

replies_to_hasan['renderedContent'] = replies_to_hasan['renderedContent'].apply(
    html.unescape).apply(remove_emojis).apply(remove_twitter_handles)


In [15]:
print(replies_to_hasan.head())


                    id                                                url   
0  1662244477254205441  https://twitter.com/AutarkistThe/status/166224...  \
1  1662242888279900160  https://twitter.com/johnsonjohnsceo/status/166...   
2  1662240517697028098  https://twitter.com/ClaudeScuba/status/1662240...   
3  1662239613983883265  https://twitter.com/finlay_glasgow/status/1662...   
4  1662237863231074304  https://twitter.com/Pray4TheBatman/status/1662...   

                 date                                         rawContent   
0 2023-05-26 23:49:01   Gays had it coming since 2015. It will get wo...  \
1 2023-05-26 23:42:42    Grifter hogs like multimillionaire Hasan beg...   
2 2023-05-26 23:33:17                    Hasan endorses child castration   
3 2023-05-26 23:29:41   “The literal using of child mannequins” there...   
4 2023-05-26 23:22:44   And? He's not part of the LGBT flag cult. Tar...   

                                     renderedContent       conversationId   
0  

Now we will create a stopwords list. Stopwords are words like "a", "and", "the", "an", so words that do not change the meaning of the sentence. We will also add Hasan's name second name, twitter handle and his twitch username. What we are doing here is called lemmatization. The more you know.


In [16]:
stopwords_list = stopwords.words('english')
stopwords.encoding('')
stopwords_list.extend(
    ['hasan', "parker", "hasanthehun", "hasanabi"])

print(stopwords_list[:5])


['i', 'me', 'my', 'myself', 'we']


This is a function that will tokenize the tweets. Tokenizing refers to the process of breaking down a text or a sentence into smaller units called tokens. Tokens are typically words, but they can also be phrases, sentences, or even individual characters, depending on the level of granularity desired. This will make our tweets easier to analyze by our NLP (natural language processing) algorithms.


In [20]:
def tokenize(text: str) -> list[str]:
    words = nltk.word_tokenize(text)
    return [word.lower() for word in words if word.isalpha()]

This function will apply our stopwords to the tweets. This is called "lemmatization". It is the process of reducing words to their base or canonical form. The lemma represents the dictionary or canonical form of a word, from which all inflected forms (such as different tenses, plurals, or derivations) are derived. It is used to reduce different forms of a word to a common base. This is because they will be treated as the same item by our algorithms. If they were different they could make some differences making our results inaccurate.


In [23]:
lemmatizer = WordNetLemmatizer()


def lemmatize(text: str) -> list[str]:
    words = [word for word in text if word not in stopwords_list]
    return [lemmatizer.lemmatize(word) for word in words]

Now we will apply this to our data.


In [24]:
hasan_tweets['rawContent'] = hasan_tweets['rawContent'].apply(
    tokenize).apply(lemmatize)

hasan_tweets['renderedContent'] = hasan_tweets['renderedContent'].apply(
    tokenize).apply(lemmatize)


In [25]:
replies_to_hasan['rawContent'] = replies_to_hasan['rawContent'].apply(
    tokenize).apply(lemmatize)

replies_to_hasan['renderedContent'] = replies_to_hasan['renderedContent'].apply(
    tokenize).apply(lemmatize)


## Analyzing

Now we will do the most interesting part. We will use some of the NLP algorithms to calculate a sentiment and add this to our dataset.


In [26]:
sid = SentimentIntensityAnalyzer()

replies_to_hasan['sentiment'] = replies_to_hasan['rawContent'].apply(
    lambda text: sid.polarity_scores(' '.join(text))['compound'])

Extract the minimum and maximum sentiments.

In [27]:
sentiment_max = replies_to_hasan['sentiment'].max()
sentiment_min = replies_to_hasan['sentiment'].min()

print(sentiment_max, sentiment_min)

0.9998 -0.9862


Let's print 5 most positive comments

In [28]:
most_positive_replies = replies_to_hasan.sort_values(by='sentiment', ascending=False)[:5]

for i in range(most_positive_replies.shape[0]):
    print(f"{i+1}. {most_positive_replies.iloc[i]['renderedContent']}")

1. ['wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow', 'wow']
2. ['believe', 'freedom', 'speech', 'believe', 'freedom', 'view', 'like', 'dictator', 'favor', 'freedom', 'speech', 'view', 'liked', 'favor', 'freedom', 'speech', 'favor', 'freedom', 'speech', 'precisely', 'view', 'despise', 'chomsky']
3. ['beautiful', 'pup', 'soon', 'saw', 'obvious', 'two', 'meant', 'best', 'friend', 'yin', 'yang', 'congrats', 'hope', 'pup', 'brings', 'joy', 'laughter', 'love', 'year', 'come']
4. ['fucking', 'love', 'shit', 'lmfao', 'love', 'tate', 'reason', 'loved', 'trump', 'office

Some of them are gibberish two of them relate to Hasan getting a dog recently and the rest relate to political topics. Hasan's crowd at it's finest.

Now for the less fun ones. Let's see what are the 5 most negative sentiments

In [29]:
most_positive_replies = replies_to_hasan.sort_values(by='sentiment', ascending=True)[:5]

for i in range(most_positive_replies.shape[0]):
    print(f"{i+1}. {most_positive_replies.iloc[i]['renderedContent']}")

1. ['ever', 'tried', 'purchase', 'firearm', 'doubt', 'know', 'nation', 'state', 'gun', 'law', 'doubt', 'rock', 'kill', 'abel', 'cain', 'mental', 'illness', 'problem', 'gun', 'problem', 'killing', 'child', 'gun', 'sick', 'person', 'messed']
2. ['thing', 'leak', 'reveal', 'crime', 'injustice', 'u', 'government', 'reveals', 'critical', 'intel', 'regarding', 'war', 'put', 'ukrainian', 'civilian', 'danger', 'since', 'difficult', 'defend', 'country', 'like', 'war', 'crime', 'leak']
3. ['saw', 'take', 'think', 'porn', 'better', 'advertiser', 'crypto', 'really', 'addiction', 'lead', 'problematic', 'behavior', 'probably', 'lead', 'lot', 'cultural', 'problem', 'better', 'global', 'currency', 'gtfoh', 'insane', 'hate', 'thing', 'insane', 'hate', 'gambling', 'stream', 'cool', 'porno', 'ad', 'hate', 'crypto', 'think', 'bad', 'believe', 'usd', 'bad', 'crypto', 'horrible', 'socialist', 'see', 'crypto', 'buffer', 'big', 'bank', 'insane', 'take']
4. ['actually', 'half', 'gun', 'death', 'suicide', 'vast

Wow, three of them relate to guns in US. One is about the war in Ukraine and the last one relates to a problem of porn addiction.